In [17]:
using Plots, DifferentialEquations, LaTeXStrings, Printf, SpecialFunctions

function explicit_ode(du, u, p, t)
    W = u[1]
    # Properly unpack parameters - this was the main issue
    Q, M, S, f₋, f₊, k₋, k₊, f, k = p

    # ========== Common Terms ==========
    p² = t^2
    Q_p² = Q + p²
    S²₃ = S^(2 / 3)
    S⁴₃ = S^(4 / 3)
    f₋k₋ = f₋ * k₋
    f₊k₊ = f₊ * k₊
    f₋²k₋² = f₋k₋^2
    f₊²k₊² = f₊k₊^2
    denom₋ = Q * Q_p² + S²₃ * f₋²k₋²
    denom₊ = Q * Q_p² + S²₃ * f₊²k₊²
    denom₋_neg = -Q * Q_p² + S²₃ * f₋²k₋²
    denom₊_neg = -Q * Q_p² + S²₃ * f₊²k₊²

    # ========== Numerator Components ==========
    main_term = p² * Q + (1 / 4) * M * p² * S²₃ * (
                             M * p² * Q_p² * S²₃ * (1 / denom₋_neg + 1 / denom₊_neg) +
                             (im * f * k * M * t * S * (
                                  -(f₋k₋ * (Q * (-p² + Q) + S²₃ * f₋²k₋²) / denom₋^2) -
                                  (f₊k₊ * (Q * (-p² + Q) + S²₃ * f₊²k₊²) / denom₊^2)
                              )) / Q_p² +
                             (2 * im * f * k * M * t^3 * S * (f₋k₋ / denom₋ + f₊k₊ / denom₊)) / Q_p²^2 -
                             (3 * im * f * k * M * t * S * (f₋k₋ / denom₋ + f₊k₊ / denom₊)) / Q_p² -
                             (M^3 * p^4 * S^(8 / 3) * (f₋k₋ + f₊k₊)^2 * (Q * Q_p² - S²₃ * f₋k₋ * f₊k₊) *
                              (Q * Q_p² + S²₃ * f₋k₋ * f₊k₊)) /
                             (denom₋_neg * denom₊_neg *
                              (S²₃ * f₋²k₋² * (4 * p²^2 * Q + 4 * Q^3 + p² * Q * (8 * Q + M^2 * S⁴₃) + 4 * Q_p² * S²₃ * f₊²k₊²) +
                               Q * (2 * Q * Q_p² * (2 * p²^2 + 2 * Q^2 + p² * (4 * Q + M^2 * S⁴₃)) +
                                    S²₃ * (4 * p²^2 + 4 * Q^2 + p² * (8 * Q + M^2 * S⁴₃)) * f₊²k₊²)))
                         )

    # W-dependent terms
    W_terms = (1 / 4) * (
        (16 * f^2 * k^2 * denom₋ * denom₊) /
        (S²₃ * f₋²k₋² * (4 * p²^2 * Q + 4 * Q^3 + p² * Q * (8 * Q + M^2 * S⁴₃) + 4 * Q_p² * S²₃ * f₊²k₊²) +
         Q * (2 * Q * Q_p² * (2 * p²^2 + 2 * Q^2 + p² * (4 * Q + M^2 * S⁴₃)) +
              S²₃ * (4 * p²^2 + 4 * Q^2 + p² * (8 * Q + M^2 * S⁴₃)) * f₊²k₊²)) +
        (1 / Q_p²^3) * f * k * (
            8 * f * k * p² * Q_p² - 8 * f * k * Q_p²^2 -
            im * M^2 * t^3 * Q_p²^2 * S^(5 / 3) * (f₋k₋ / denom₋ + f₊k₊ / denom₊)
        ) * W -
        (4 * f^2 * k^2 * denom₋ * denom₊) /
        (S²₃ * f₋²k₋² * (4 * p²^2 * Q + 4 * Q^3 + p² * Q * (8 * Q + M^2 * S⁴₃) + 4 * Q_p² * S²₃ * f₊²k₊²) +
         Q * (2 * Q * Q_p² * (2 * p²^2 + 2 * Q^2 + p² * (4 * Q + M^2 * S⁴₃)) +
              S²₃ * (4 * p²^2 + 4 * Q^2 + p² * (8 * Q + M^2 * S⁴₃)) * f₊²k₊²)) * W^2)

    # ========== Denominator ==========
    denominator = (4 * f^2 * k^2 * t * denom₋ * denom₊) /
                  (S²₃ * f₋²k₋² * (4 * p²^2 * Q + 4 * Q^3 + p² * Q * (8 * Q + M^2 * S⁴₃) + 4 * Q_p² * S²₃ * f₊²k₊²) +
                   Q * (2 * Q * Q_p² * (2 * p²^2 + 2 * Q^2 + p² * (4 * Q + M^2 * S⁴₃)) +
                        S²₃ * (4 * p²^2 + 4 * Q^2 + p² * (8 * Q + M^2 * S⁴₃)) * f₊²k₊²))

    # ========== Final Derivative Calculation ==========
    du[1] = (main_term + W_terms) / denominator
end

# Parameters setup
Q = 1e-2
M = 1e-2
S = 1e4
f_minus = 1.0
f_plus = 1.0
k_minus = 1.0
k_plus = 1.0
f = 1.0
k = 1.0

params = (Q, M, S, f_minus, f_plus, k_minus, k_plus, f, k)

# Time span - integrate from small to large p (t in ODEProblem)
tspan = (1e-7, 20.0)

function solve_and_compute_Δ(p1, p2)
    # Initial condition - ensure proper complex number creation
    initial_imag = (-4 * p2 * first(tspan)) / (p1^2 * μ)
    w0 = [complex(0.0, initial_imag)]  # Correct complex array initialization

    # Create ODE problem with properly structured parameters
    prob = ODEProblem(explicit_ode, w0, tspan, (p1, p2, S, f_minus, f_plus, k_minus, k_plus, f, k))

    try
        # Try Vern7 first for complex numbers
        sol = solve(prob, Vern7(), reltol=1e-6, abstol=1e-6, maxiters=1e5)

        # Fallback to Tsit5 if needed
        if !SciMLBase.successful_retcode(sol.retcode)
            sol = solve(prob, Tsit5(), reltol=1e-6, abstol=1e-6, maxiters=1e5)
        end

        if !SciMLBase.successful_retcode(sol.retcode)
            @warn "Solver failed for p1=$p1, p2=$p2 with retcode $(sol.retcode)"
            return complex(NaN, NaN)
        end

        t_min = minimum(sol.t)
        W_prime = sol(t_min, Val{1})[1]
        Δhat = π / W_prime
        return Δhat

    catch e
        @warn "Complete failure for p1=$p1, p2=$p2: $(sprint(showerror, e))"
        return complex(NaN, NaN)
    end
end

# Test with scaled parameters
test_vals = 10.0 .^ range(-4, -2, length=5)
results = [solve_and_compute_Δ(Q, M_val) for M_val in test_vals]
println("Test results: ", results)

┌ Warning: Complete failure for p1=0.01, p2=0.0001: MethodError: no method matching ^(::NTuple{9, Float64}, ::Int64)
│ The function `^` exists, but no method is defined for this combination of argument types.
│ 
│ Closest candidates are:
│   ^(!Matched::Float32, ::Integer)
│    @ Base math.jl:1228
│   ^(!Matched::Float16, ::Integer)
│    @ Base math.jl:1234
│   ^(!Matched::BigFloat, ::Union{Int16, Int32, Int64, Int8})
│    @ Base mpfr.jl:717
│   ...
└ @ Main In[17]:108
┌ Warning: Complete failure for p1=0.01, p2=0.00031622776601683794: MethodError: no method matching ^(::NTuple{9, Float64}, ::Int64)
│ The function `^` exists, but no method is defined for this combination of argument types.
│ 
│ Closest candidates are:
│   ^(!Matched::Float32, ::Integer)
│    @ Base math.jl:1228
│   ^(!Matched::Float16, ::Integer)
│    @ Base math.jl:1234
│   ^(!Matched::BigFloat, ::Union{Int16, Int32, Int64, Int8})
│    @ Base mpfr.jl:717
│   ...
└ @ Main In[17]:108
┌ Warning: Complete failure for p1=0

Test results: ComplexF64[NaN + NaN*im, NaN + NaN*im, NaN + NaN*im, NaN + NaN*im, NaN + NaN*im]
